In [2]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np
import pprint

In [11]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")


cfg = Config.fromfile('./deformable_detr_twostage_refine_r50_16X2_50e_coco.py')

root='../../../../dataset/'

cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'

cfg.data.test.test_mode = True

cfg.work_dir = './work_dirs/deformable_detr_twostage_refine_r50_16X2_50e_coco'

epoch = 'latest'


In [12]:
pprint.pprint(cfg.data.test)

{'ann_file': '../../../../dataset/test.json',
 'classes': ('General trash',
             'Paper',
             'Paper pack',
             'Metal',
             'Glass',
             'Plastic',
             'Styrofoam',
             'Plastic bag',
             'Battery',
             'Clothing'),
 'img_prefix': '../../../../dataset/',
 'pipeline': [{'type': 'LoadImageFromFile'},
              {'flip': False,
               'img_scale': (1333, 800),
               'transforms': [{'keep_ratio': True, 'type': 'Resize'},
                              {'type': 'RandomFlip'},
                              {'mean': [123.675, 116.28, 103.53],
                               'std': [58.395, 57.12, 57.375],
                               'to_rgb': True,
                               'type': 'Normalize'},
                              {'size_divisor': 1, 'type': 'Pad'},
                              {'keys': ['img'], 'type': 'ImageToTensor'},
                              {'keys': ['img'], 'type':

In [13]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.21s)
creating index...
index created!


In [14]:
print(data_loader.dataset)
print(dataset)


CocoDataset Test dataset with number of images 4871, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 0     | 1 [Paper]     | 0     | 2 [Paper pack]  | 0     | 3 [Metal]   | 0     | 4 [Glass]    | 0     |
| 5 [Plastic]       | 0     | 6 [Styrofoam] | 0     | 7 [Plastic bag] | 0     | 8 [Battery] | 0     | 9 [Clothing] | 0     |
|                   |       |               |       |                 |       |             |       |              |       |
| -1 background     | 4871  |               |       |                 |       |             |       |              |       |
+-------------------+-------+---------------+----

In [15]:
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

load checkpoint from local path: ./work_dirs/deformable_detr_twostage_refine_r50_16X2_50e_coco/latest.pth


In [16]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

[                                ] 3/4871, 7.5 task/s, elapsed: 0s, ETA:   646s

/data/ephemeral/home/baseline/mmdetection/mmdet/models/utils/positional_encoding.py:81: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature**(2 * (dim_t // 2) / self.num_feats)
/data/ephemeral/home/baseline/mmdetection/mmdet/models/utils/transformer.py:883: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = tempe

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 15.6 task/s, elapsed: 312s, ETA:     0s

In [17]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10

for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=img_ids[i]))[0]
    
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.31805477 244.37642 684.1408 297.78598 751....,test/0000.jpg
1,0 0.5475998 748.8038 655.8013 897.96204 987.28...,test/0001.jpg
2,0 0.50443953 89.39138 273.55756 423.73456 602....,test/0002.jpg
3,0 0.032460466 0.0 0.9500427 912.6985 817.7748 ...,test/0003.jpg
4,0 0.45714805 191.91547 251.0104 873.99384 769....,test/0004.jpg
